# Retrieval Augmented Generation

Overview:
- Introduction to RAG with LlamaIndex
- Data ingestion
  - PDF
  - Web pages
  - Code
- Data splitting
  - Token splitting
  - Sentence splitting
  - Structured data splitting
  - Semantic chunking
- Vectorization
  - Embeddings
  - Vector storage
- Retrieval
  - Keyword search
  - Vector search
  - Hybrid search
- Advanced methods
  - Query rewriting
  - Multi-hop retrieval

# Introduction to RAG with LlamaIndex

LlamaIndex is a library for working with large language models.
One of its main strengths is its ability to ingest documents into a vector index and use them to answer questions.
This is known as Retrieval Augmented Generation (RAG).

To start, we will use a low-code, high-level abstraction to build a basic PDF question-answering system.
We will read in PDFs, split them into chunks, embed them, and store them in a vector database.
Then, we will use an abstraction known as a `QueryEngine` that implements RAG to answer questions about the documents.

In [5]:
# If we're in colab, use userdata to get the OPENAI_API_KEY
import os
from rich import print
from pathlib import Path

try:
    from google.colab import userdata
    print("Colab detected - setting up environment")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    %pip install llama-index \
        llama-index-readers-web \
        thefuzz \
        gradio \
        chromadb \
        llama-index-embeddings-huggingface \
        llama-index-vector-stores-chroma \
        llama-index-llms-gemini \
        marker-pdf \
        bm25s \
        PyStemmer
        
except:
    print("Not in colab - using local environment variables.")
    from dotenv import load_dotenv
    load_dotenv("../.env")


Not in colab - using local environment variables.

In [6]:
import os
import requests

# Create data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the PDF file
pdf_url = "https://arxiv.org/pdf/2407.21783"
pdf_path = os.path.join(data_dir, "2407.21783.pdf")

if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded PDF to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")


PDF already exists at data/2407.21783.pdf

In [7]:
from llama_index.core.readers import SimpleDirectoryReader
documents = SimpleDirectoryReader(data_dir).load_data()

In [78]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from torch.backends.mps import is_available as is_mps_available
from torch.cuda import is_available as is_cuda_available

if is_mps_available():
    device = "mps"
elif is_cuda_available():
    device = "cuda"
else:
    device = "cpu"

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", device=device)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [65]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm # set gpt-4o-mini as the default llm

query_engine = index.as_query_engine(llm=llm)

: 

In [ ]:
response = query_engine.query("How many new Llama models models are mentioned in the paper?")

: 

In [ ]:
print(response.response)

: 

## Exercise: Create a Gradio interface for a question-answering system

Your goal in this exercise is to create a Gradio interface for a question-answering system.
Your application should:
- Use the query engine created above to answer questions about the uploaded PDF
- Display the question and answer in the UI

If you need a challenge:
- Use the `gr.File` component to allow the user to upload ANY pdf and ask question about it.


In [ ]:
import gradio as gr

In [ ]:
from tempfile import TemporaryDirectory
import os
from pathlib import Path

def ingest_documents(file_obj):
    """
    Process an uploaded PDF file and create a query engine for it.
    
    Args:
        file_obj: Gradio file upload object containing the PDF
        
    Returns:
        QueryEngine: Configured query engine for the ingested document
    """
    if file_obj is None:
        return None
        
    with TemporaryDirectory() as temp_dir:
        file_path = os.path.join(temp_dir, 'tmp.pdf')
        # Get the file bytes from the Gradio upload object
        file_bytes = open(file_obj, "rb").read()
        
        with open(file_path, "wb") as f:
            f.write(file_bytes)
            
        documents = SimpleDirectoryReader(temp_dir).load_data()
        llm = OpenAI(model="gpt-4o-mini")
        index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
        query_engine = index.as_query_engine(llm=llm)
        
    return query_engine, None, Path(file_obj.name).name

def predict(query_engine, question):
    return query_engine.query(question)


In [ ]:
with gr.Blocks() as demo:
    query_engine = gr.State()
    gr.Markdown("## RAG Demo: Question answering with a PDF")
    with gr.Row():
        with gr.Column(scale=1):   
            file_upload = gr.File(label="Upload a PDF file", file_types=[".pdf"], file_count="single")
            submit_button = gr.Button("Submit")
            pdf_name = gr.Textbox(label="You are asking about...")
        with gr.Column(scale=3):
            input = gr.Textbox(label="Enter a question")
            output = gr.Textbox(label="Answer")

    submit_button.click(fn=ingest_documents, inputs=file_upload, outputs=[query_engine, file_upload, pdf_name])
    input.submit(fn=predict, inputs=[query_engine, input], outputs=output)

demo.launch()

# Data ingestion

Data often comes in many different formats.
It may come in the form of a PDF, a web page, a code file, etc.
We may need some specific processing pipelines to extract the text from these documents, split them correctly, and vectorize them.

Luckily, LlamaIndex (and other libraries) provide lots of built-in and add-on tools to help you ingest almost any data type.
Instead of loading a PDF, let's load a web page instead.
We will use one of the classes provided by [`llama-index-readers-web`](https://llamahub.ai/l/readers/llama-index-readers-web?from=readers) to load data from a web page.

In this section, we will:
- Load a web page as Markdown
- Split it into chunks following the structured format of the Markdown
- Embed the chunks
- Store the chunks in a vector database
- Create a query engine from the vector database and use it to answer a question


In [79]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


In [80]:
web_docs = SimpleWebPageReader(html_to_text=True).load_data(['https://en.wikipedia.org/wiki/Wikipedia'])

In [82]:
# chromadb.EphemeralClient().delete_collection("wikipedia")

In [83]:
collection = chromadb.EphemeralClient().create_collection("wikipedia", get_or_create=True)
vector_store = ChromaVectorStore(collection)

In [84]:
pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser.from_defaults(),
        embed_model,
    ], 
    vector_store=vector_store
)

In [85]:
nodes = pipeline.run(documents=web_docs)

In [86]:
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [87]:
llm = OpenAI(model="gpt-4o-mini")
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many languages there exactly? Quote the exact text as well.")
print(response.response)

There are currently 339 language editions of Wikipedia.

In [88]:
# Use fuzzywuzzy to find the closest match in the source text
from thefuzz import fuzz, process
# Get the top matching line of text from the source_text_quote
top_match, match_score = process.extractOne(response.response, response.source_nodes[0].text.splitlines(), scorer=fuzz.ratio)
assert top_match in response.source_nodes[0].text
print(f"Quote from source: '{top_match}'")

Quote from source: 'There are currently 339 language editions of Wikipedia (also called _language'

## OCR

There are times where the PDF is not in a format that can be easily read into text.
In these cases, we will need to use optical character recognition (OCR) to convert the images to text.
There are many libraries and cloud services that can do this, but for this example, we will use the `marker` library since our example document is a PDF.
We will then ingest the text into LlamaIndex and use it to answer a question.

In [36]:
from marker.convert import convert_single_pdf
from marker.models import load_all_models
from IPython.display import display, Markdown

In [ ]:
fpath = "data/2407.21783.pdf"
model_lst = load_all_models()
full_text, images, out_meta = convert_single_pdf(fpath, model_lst)

In [ ]:
display(Markdown(full_text[:5000]))

In [14]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core.schema import Document

In [34]:
markdown_parser = MarkdownElementNodeParser.from_defaults()

In [ ]:
nodes = await markdown_parser.aget_nodes_from_documents([Document(text=full_text)])

In [ ]:
print(nodes[2].text)

# Retrieval

## Starting simple: bm25

BM25 is a simple retrieval method that uses the BM25 algorithm to score the relevance of each document to the query.
The BM25 algorithm is a probabilistic retrieval model that uses the term frequency and inverse document frequency of the query terms to score the relevance of each document.
You should be familiar with the basic idea of tf-idf from your NLP class - you can think of BM25 as a generalization of tf-idf that takes into account more factors.

In [63]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle
from llama_index.core.query_engine import RetrieverQueryEngine
import bm25s
import Stemmer
from typing import List

In [70]:
class BM25Retriever(BaseRetriever):
    def __init__(self, index: VectorStoreIndex, stemmer: Stemmer.Stemmer = Stemmer.Stemmer("english"), k: int = 10):
        self.index = index
        self.stemmer = stemmer
        self.k = k
        self._ids = []
        self._corpus = []
        self._setup()

    def _tokenize(self, text: str):
        return bm25s.tokenize(text, stopwords="en", stemmer=self.stemmer)

    def _setup(self):
        for doc_id, doc in self.index.docstore.docs.items():
            self._ids.append(doc_id)
            self._corpus.append(doc.text)
        self._bm25 = bm25s.BM25()
        corpus_tokens = self._tokenize(self._corpus)
        self._bm25.index(corpus_tokens)

    def _get_node_from_text(self, text: str, score: float) -> NodeWithScore:
        text_idx = self._corpus.index(text)
        id = self._ids[text_idx]
        node = self.index.docstore.get_node(id)
        return NodeWithScore(node=node, score=score)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query_tokens = self._tokenize(query_bundle.query_str)
        # scores is a list of tuples (text, score)
        documents, scores = self._bm25.retrieve(query_tokens, corpus=self._corpus, k=self.k)
        
        nodes_with_scores = []
        for text, score in zip(documents[0], scores[0]):
            node_with_score = self._get_node_from_text(text, float(score))
            nodes_with_scores.append(node_with_score)
            
        return nodes_with_scores

In [71]:
bm25 = BM25Retriever(index, k=2)

Split strings:   0%|          | 0/135 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/135 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/135 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/135 [00:00<?, ?it/s]

In [76]:
for node in bm25.retrieve("What is the main idea of the paper?"):
    print(f"Score: {node.score:.4f}\nText:\n{node.node.text[:500]}...")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Score: 2.4542
Text:
Wei Liu, Weihao Zeng, Keqing He, Yong Jiang, and Junxian He. What makes good data for alignment? a comprehensive
study of automatic data selection in instruction tuning, 2024c.https://arxiv.org/abs/2312.15685.
Yinhan Liu, Myle Ott, Naman Goyal, Jingfei Du, Mandar Joshi, Danqi Chen, Omer Levy, Mike Lewis, Luke Zettlemoyer,
and Veselin Stoyanov. Roberta: A robustly optimized bert pretraining approach.arXiv preprint arXiv:1907.11692,
2019a.
Yinhan Liu, Myle Ott, Naman Goyal, Jingfei Du, Mandar Josh...

Score: 2.2030
Text:
The Llama 3 Herd of Models
Llama Team, AI @ Meta 1
1A detailed contributor list can be found in the appendix of this paper.
Modern artificial intelligence (AI) systems are powered by foundation models. This paper presents a
new set of foundation models, called Llama 3. It is a herd of language models that natively support
multilinguality, coding, reasoning, and tool usage. Our largest model is a dense Transformer with
405B parameters and a context window of up to 128K tokens. This paper presents...

## Dense retrieval (vector search)

BM25 is a simple and fast method that depends on word matching.
But if we want to do more complex retrieval, we can use dense retrieval.
We represent both our query and documents as vectors and use a similarity metric to find the most relevant documents.
This is what's been going on under the hood in the previous examples using `VectorStoreIndex`.

Since most of the mechanics are taken care for us under the hood, let's examine what goes on under the hood.

In [121]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt


In [126]:
sample_documents = [
    "My favorite type of dog is a golden retriever.",
    "I like to eat pizza with my friends.",
    "I like to go to the gym in the morning.",
    "I like to play basketball with my friends.",
]

embeddings = np.array(embed_model.get_text_embedding_batch(sample_documents))

query = "What do I like to do with friends?"
query_embedding = np.array(embed_model.get_text_embedding(query))

In [129]:
cosine_similarity(query_embedding.reshape(1, -1), embeddings).squeeze()

array([0.40551496, 0.65599524, 0.45809953, 0.64565545])

## Hybrid search: query rewriting and reciprocal ranking

Sometimes, you may want several methods of searching over your data, then combining the results.
This is known as hybrid search.

Haveing multiple retrievers may not mean having separate objects - we may just have multiple queries.
In this example, we'll use an LLM to rewrite our query into multiple queries, then use a dense retriever to find the most relevant documents.
Finally, we'll use reciprocal ranking to re-rank the results.


In [130]:
from llama_index.core.retrievers import QueryFusionRetriever

In [148]:
dense_retriever = index.as_retriever(similarity_top_k=5)
hybrid_retriever = QueryFusionRetriever(
    [dense_retriever],
    num_queries=3,
    use_async=False,
    mode='reciprocal_rerank',
    verbose=True
)

In [156]:
result = hybrid_retriever.retrieve("How many languages does wikipedia have?")

Generated queries:
How many languages are available on Wikipedia?
What is the total number of Wikipedia language editions?


In [157]:
print(result[0].text)

## Language editions

Main article: [List of Wikipedias](/wiki/List_of_Wikipedias "List of
Wikipedias")

**Distribution of the 63,955,872 articles in different language editions (as
of November 17, 2024)**[W 42]

[English](/wiki/English_Wikipedia "English Wikipedia") (10.8%)

[Cebuano](/wiki/Cebuano_Wikipedia "Cebuano Wikipedia") (9.6%)

[German](/wiki/German_Wikipedia "German Wikipedia") (4.6%)

[French](/wiki/French_Wikipedia "French Wikipedia") (4.1%)

[Swedish](/wiki/Swedish_Wikipedia "Swedish Wikipedia") (4.1%)

[Dutch](/wiki/Dutch_Wikipedia "Dutch Wikipedia") (3.4%)

[Russian](/wiki/Russian_Wikipedia "Russian Wikipedia") (3.1%)

[Spanish](/wiki/Spanish_Wikipedia "Spanish Wikipedia") (3.1%)

[Italian](/wiki/Italian_Wikipedia "Italian Wikipedia") (3%)

[Polish](/wiki/Polish_Wikipedia "Polish Wikipedia") (2.6%)

[Egyptian Arabic](/wiki/Egyptian_Arabic_Wikipedia "Egyptian Arabic Wikipedia")
(2.5%)

[Chinese](/wiki/Chinese_Wikipedia "Chinese Wikipedia") (2.3%)

[Japanese](/wiki/Japanese_Wikipedia "Japanese Wikipedia") (2.2%)

[Ukrainian](/wiki/Ukrainian_Wikipedia "Ukrainian Wikipedia") (2.1%)

[Vietnamese](/wiki/Vietnamese_Wikipedia "Vietnamese Wikipedia") (2%)

[Waray](/wiki/Waray_Wikipedia "Waray Wikipedia") (2%)

[Arabic](/wiki/Arabic_Wikipedia "Arabic Wikipedia") (1.9%)

[Portuguese](/wiki/Portuguese_Wikipedia "Portuguese Wikipedia") (1.9%)

[Persian](/wiki/Persian_Wikipedia "Persian Wikipedia") (1.6%)

[Catalan](/wiki/Catalan_Wikipedia "Catalan Wikipedia") (1.2%)

Other (31.9%)

There are currently 339 language editions of Wikipedia (also called _language
versions_ , or simply _Wikipedias_). As of November 2024, the six largest, in
order of article count, are the [English](/wiki/English_Wikipedia "English
Wikipedia"), [Cebuano](/wiki/Cebuano_Wikipedia "Cebuano Wikipedia"),
[German](/wiki/German_Wikipedia "German Wikipedia"),
[French](/wiki/French_Wikipedia "French Wikipedia"),
[Swedish](/wiki/Swedish_Wikipedia "Swedish Wikipedia"), and
[Dutch](/wiki/Dutch_Wikipedia "Dutch Wikipedia") Wikipedias.[W 43] The second
and fifth-largest Wikipedias owe their position to the article-creating bot
[Lsjbot](/wiki/Lsjbot "Lsjbot"), which as of
2013[](https://en.wikipedia.org/w/index.php?title=Wikipedia&action=edit)
had created about half the articles on the [Swedish
Wikipedia](/wiki/Swedish_Wikipedia "Swedish Wikipedia"), and most of the
articles in the [Cebuano](/wiki/Cebuano_Wikipedia "Cebuano Wikipedia") and
[Waray Wikipedias](/wiki/Waray_Wikipedia "Waray Wikipedia"). The latter are
both languages of the [Philippines](/wiki/Philippines "Philippines").

In addition to the top six, twelve other Wikipedias have more than a million
articles each ([Russian](/wiki/Russian_Wikipedia "Russian Wikipedia"),
[Spanish](/wiki/Spanish_Wikipedia "Spanish Wikipedia"),
[Italian](/wiki/Italian_Wikipedia "Italian Wikipedia"),
[Polish](/wiki/Polish_Wikipedia "Polish Wikipedia"), [Egyptian
Arabic](/wiki/Egyptian_Arabic_Wikipedia "Egyptian Arabic Wikipedia"),
[Chinese](/wiki/Chinese_Wikipedia "Chinese Wikipedia"),
[Japanese](/wiki/Japanese_Wikipedia "Japanese Wikipedia"),
[Ukrainian](/wiki/Ukrainian_Wikipedia "Ukrainian Wikipedia"),
[Vietnamese](/wiki/Vietnamese_Wikipedia "Vietnamese Wikipedia"),
[Waray](/wiki/Waray_Wikipedia "Waray Wikipedia"),
[Arabic](/wiki/Arabic_Wikipedia "Arabic Wikipedia"), and
[Portuguese](/wiki/Portuguese_Wikipedia "Portuguese Wikipedia")), seven more
have over 500,000 articles ([Persian](/wiki/Persian_Wikipedia "Persian
Wikipedia"), [Catalan](/wiki/Catalan_Wikipedia "Catalan Wikipedia"),
[Indonesian](/wiki/Indonesian_Wikipedia "Indonesian Wikipedia"),
[Serbian](/wiki/Serbian_Wikipedia "Serbian Wikipedia"),
[Korean](/wiki/Korean_Wikipedia "Korean Wikipedia"),
[Norwegian](/wiki/Norwegian_Wikipedia "Norwegian Wikipedia"), and
[Turkish](/wiki/Turkish_Wikipedia "Turkish Wikipedia")), 44 more have over
100,000, and 82 more have over 10,000.[W 44][W 43] The largest, the English
Wikipedia, has over 6.9 million articles. As of January